## Детекция названий препаратов

Положите данные в папку с ноутбуком.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
#import time
import re
#import nltk
import sklearn
from collections import Counter
from nltk import word_tokenize
import _pickle

In [2]:
data = pd.read_csv('clitical_trials_100000.tsv', delimiter='\t', encoding='utf-8').values

In [3]:
#Возвращает названия препаратов из строки таблицы
def words_by_coord (inform):
    coords = re.findall("(\d+, \d+)", inform[0][1:-1])
    drugs_coord = []
    names = []
    for i in coords:
        drugs_coord.append(re.split(", ", i))
    for k in drugs_coord:
        names.append(inform[1][int(k[0]):int(k[1])])
    return names

In [10]:
#Возвращает список всех названий препаратов из таблицы
def extract_drugs(_data):
    _buf = []
    for i in _data:
        _buf += words_by_coord(i)
    return _buf

In [11]:
def my_tokenizer(sentence):
    return list(filter(lambda x: x != ',', word_tokenize(sentence)))

In [12]:
#Подсчет количества слов
def word_counter (_data):
    _num = 0
    for i in _data:
        s = 0
        _tkzed = word_tokenize(i[1])
        for q in _tkzed:
            if (len(q) <= 3):
                s +=1
        _num += len(_tkzed) - s
    return _num

In [13]:
#Преобразование таблицы в новую, где каждая строка - одно слово, номер его текста и булево значение препарат - не прпеарат
def create_data(_data, _len):
    drug_array = np.empty(( _len), dtype=bool)
    sentence_array = np.empty((_len), dtype=int)
    word_array = np.empty((_len), dtype='<U16')
    counter = 0
    for i, row in enumerate(_data):
        drugs_in_t = set(extract_drugs([row]))
        for word in word_tokenize(row[1]):
            if (len(word) > 3):
                sentence_array[counter] = i
                word_array[counter] = word
                drug_array[counter] = not drugs_in_t.isdisjoint(set([word]))
                counter += 1
    return pd.DataFrame({'word':word_array, 'sentence':sentence_array, 'drug':drug_array})

In [14]:
#Создание таблицы слово - метка
def create_data_for_FT(_data, _len):
    drug_array = np.empty(( _len), dtype=bool)
    word_array = np.empty((_len), dtype='<U16')
    counter = 0
    for row in _data:
        drugs_in_t = set(extract_drugs([row]))
        for word in word_tokenize(row[1]):
            if (len(word) > 3):
                word_array[counter] = word
                drug_array[counter] = not drugs_in_t.isdisjoint(set([word]))
                counter += 1
    return pd.DataFrame({'word':word_array, 'drug':drug_array})

Создадим список всех известных препаратов.

In [11]:
buf = extract_drugs(data)
known_drugs = set(buf)
known_drugs

{'dalotuzumab',
 'len',
 '6th',
 'lipopeptide',
 'double',
 'suicide',
 'z-vad',
 'motif',
 'dhfr',
 'tnf-alpha-dependent-inos-independent',
 'lhrha',
 'enhance',
 'primary',
 'cgy',
 'anti-cancer',
 'egfr-tyrosine-kinase',
 'terminal',
 'bct',
 'tag',
 'dox',
 'urals',
 'premedication',
 'estramustine',
 'r-cvp',
 'live-donor',
 'tomodirect',
 'duel',
 'kilobase',
 'radioimmunchemotherapy',
 '26s',
 'nsaid',
 'bid',
 'folic',
 'physiokinesic',
 'nras',
 'cdh1',
 'ril-21',
 'corporeal',
 'gamma-secreting',
 'ponatinib',
 'prescribe',
 'non-cross-resistant',
 'lenvantinib',
 'alfa-2b',
 'wbrt',
 'ang',
 'target-dependent',
 'mmc',
 '51',
 'alkylator-based',
 'cddp-containing',
 'c-myc',
 'cornea',
 'trf',
 '-booster',
 'well-described',
 '?-2b',
 'convenient',
 'anti-id',
 'varicella',
 'anti-androgen',
 'calpeptin',
 'fibroblasts.this',
 'das-driven',
 'gcv',
 'mapk-activating',
 'intracoronary',
 'clodronate',
 'el4-b7.1',
 'soak',
 'brathytherapy',
 'mir-21-3p',
 'iort',
 'monophosph

Видим, что очень много странных меток. Посмотрим на частотность.

In [14]:
print (Counter(buf).most_common(50))

[('therapy', 61913), ('chemotherapy', 60890), ('radiotherapy', 26939), ('inhibitor', 20611), ('cisplatin', 12169), ('bevacizumab', 8995), ('dexamethasone', 8328), ('transplantation', 7956), ('paclitaxel', 7583), ('calcium', 6545), ('docetaxel', 6446), ('rituximab', 6365), ('il-10', 6242), ('gemcitabine', 6023), ('doxorubicin', 5877), ('5', 5314), ('fu', 4989), ('radiation', 4850), ('cyclophosphamide', 4795), ('curcumin', 4571), ('monotherapy', 4550), ('immunotherapy', 4515), ('imatinib', 4514), ('methotrexate', 4509), ('tamoxifen', 4385), ('sorafenib', 4363), ('brachytherapy', 4153), ('fluorouracil', 4151), ('carboplatin', 4070), ('chemoradiotherapy', 3909), ('trastuzumab', 3803), ('vaccine', 3613), ('prednisone', 3383), ('erlotinib', 3330), ('capecitabine', 3301), ('oxaliplatin', 3295), ('metformin', 3245), ('ranibizumab', 3204), ('everolimus', 3063), ('cetuximab', 3015), ('cell', 2969), ('irinotecan', 2959), ('bortezomib', 2955), ('sunitinib', 2943), ('and', 2884), ('kinase', 2876), 

Будем считать, что это ошибки и в будущем подумаем как с ними работать. Можно попытаться выкинуть самые частотные ошибки вручную, но для постановки бейслайна это не нужно.

Общее количество слов - 25318985


## Постановка задачи.
Будем пытаться симулировать реальную проблему, а именно: на вход поступают тексты, а мы ищем в них названия препаратов. Таким образом получаем задачу бинарной классификации слов.   
В качестве метрики выберем f1 score, т.к. размеры классов 'название препарата' - 'обычное слово' сильно несбалансированы.
В последующем работать будем с делением по предложениям.  
Начнем с решения "в лоб".
Составим множество всех названий препаратов. Не будем делать поправку на то, сколько раз слово встречается в исходных данных. 
Лематизацию и удаление стоп слов проводить не будем, для ускорения обработки.

In [9]:
data_train, data_test = sklearn.model_selection.train_test_split(data, test_size=0.3, random_state=0)
data_train.shape[0]

70000

#### Долго.
Вычисляем количество слов в тестовой и тренировочной выборке.

In [11]:
train_num = word_counter(data_train)

In [12]:
train_num

9947779

9947779

In [11]:
test_num = word_counter(data_test)
test_num

4272307

4272307

Преобразовываем исходные данные в более удобные для обработки.

In [17]:
transform_data_train = create_data(data_train, 9947779)

In [14]:
transform_data_train.tail(10)

,word,sentence,drug
9947769,second,69999,False
9947770,disease,69999,False
9947771,into,69999,False
9947772,remission,69999,False
9947773,suddenly,69999,False
9947774,sepsis,69999,False
9947775,after,69999,False
9947776,fourth,69999,False
9947777,chemotherapy,69999,True
9947778,cycle,69999,False


In [15]:
transform_data_test = create_data(data_test, 5313143)

In [16]:
def My_predictor(data_train, data_test):
    drug_set = set(data_train.word.values[data_train.drug.values])
    y_pred = np.zeros((data_test.shape[0]))
    for i, row in enumerate(data_test.values):
        if (not drug_set.isdisjoint(set([row[0]]))):
            y_pred[i] = 1
    return y_pred

In [17]:
y_pred = My_predictor(transform_data_train, transform_data_test[['word', 'sentence']])

In [18]:
print(sklearn.metrics.classification_report(y_true=transform_data_test.drug.values, y_pred=y_pred))

              precision    recall  f1-score   support

       False       1.00      0.53      0.69   5108258
        True       0.08      0.98      0.14    204885

   micro avg       0.55      0.55      0.55   5313143
   macro avg       0.54      0.75      0.42   5313143
weighted avg       0.96      0.55      0.67   5313143



Ожидаемо видим маленький precision и большой recall.  
Можно попытаться улучшить модель - проведя OneHotEncoding и обучив на этих словах какую-нибудь линейную модель. Это позволит хорошо предсказывать известные названия препаратов, которые встречаются часто, но будет очень плохо работать с новыми, потому что вектор признаков таких слов будет состоять из одних нулей. 
Попробуем построить такую модель, а затем начнем добавлять признаки, что позволит модели начать находить неизвестные ей названия.

In [19]:
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')

In [22]:
train_one_hot = encoder.fit_transform(transform_data_train['word'].values.reshape((-1,1)))

In [21]:
test_one_hot = encoder.transform(transform_data_test['word'].values.reshape((-1,1)))

In [ ]:
log_reg = sklearn.linear_model.LogisticRegression(class_weight='balanced')
log_reg.fit(train_one_hot, transform_data_train['drug'].values * 1)

/home/nariman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
file = open('Classifier.model', 'wb')
_pickle.dump(log_reg, file)
file.close()

In [23]:
print(sklearn.metrics.classification_report(y_pred=log_reg.predict(test_one_hot), y_true=transform_data_test['drug'].values * 1))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98   5108258
           1       0.45      0.91      0.60    204885

   micro avg       0.95      0.95      0.95   5313143
   macro avg       0.72      0.93      0.79   5313143
weighted avg       0.97      0.95      0.96   5313143



Заметное улучшение, как и предсказывалось.

#### FastText
Теперь воспользуемся fasttext. Использование н-грам должно позволить нам выявлять не только особенности контекста названий препаратов, но и общее строение их названий.

In [13]:
from gensim.models import FastText

In [25]:
train_sentences = []
for row in data_train:
    for i in re.split(' \. ', row[1]):
        train_sentences.append(my_tokenizer(i))

In [ ]:
model = FastText(train_sentences, size = 200, window = 5, min_count = 2)

In [ ]:
model.save('FastText.model')

In [14]:
model = FastText.load('FastText.model')

Не выходит обработать все файлы за один подход - фасттекст занимает много памяти. Будем делать это частями.

In [17]:
data_train_for_FT = create_data_for_FT(data_train, 9947779)

In [11]:
data_test_for_FT = create_data_for_FT(data_test, 4272307)

Сначала с вектором признаков.

In [ ]:
for i in range(100):
    print(i)
    data_train_for_FT_part = data_train_for_FT[99477*i:99477*(i+1)]
    for_dump = np.empty((99477, 200))
    for count, word in enumerate(data_train_for_FT_part.word):
        for_dump[count] = model.wv[word]
    file = open('FF_emb_train'+str(i)+'.dat', 'wb')
    _pickle.dump(for_dump, file)
    file.close()
data_train_for_FT_part = data_train_for_FT[9947700:]
for_dump = np.empty((79, 200))
for i, word in enumerate(data_train_for_FT_part.word):
    for_dump[i] = model.wv[word]
file = open('FF_emb_train'+str(100)+'.dat', 'wb')
_pickle.dump(for_dump, file)
file.close()

In [15]:
for i in range(30):
    print (i)
    data_test_for_FT_part = data_test_for_FT[42723*i:42723*(i+1)]
    for_dump = np.empty((42723, 200))
    for count, word in enumerate(data_test_for_FT_part.word):
        for_dump[count] = model.wv[word]
    file = open('FF_emb_test'+str(i)+'.dat', 'wb')
    _pickle.dump(for_dump, file)
    file.close()
data_test_for_FT_part = data_test_for_FT[4272300:]
for_dump = np.empty((7, 200))
for i, word in enumerate(data_test_for_FT_part.word):
    for_dump[i] = model.wv[word]
file = open('FF_emb_test'+str(100)+'.dat', 'wb')
_pickle.dump(for_dump, file)
file.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


А теперь и меток

In [ ]:
y_train = data_train_for_FT.drug.values*1
y_test  = data_test_for_FT.drug.values*1[:42723*29]
file = open('y_train.txt', 'wb')
_pickle.dump(y_train, file)
file.close()
file = open('y_test.txt', 'wb')
_pickle.dump(y_test, file)
file.close()

In [2]:
X_train = np.empty((0,200))
for i in range(10):
    print(i)
    file = open ('FF_emb_train'+str(i)+'.dat', 'rb')
    adder = _pickle.load(file)
    X_train = np.vstack((X_train, adder))
    file.close()

0
1
2
3
4
5
6
7
8
9


In [3]:
file = open('X_train_0_10.dat', 'wb')
_pickle.dump(X_train, file)
file.close()

In [3]:
file = open('X_train_0_10.dat', 'rb')
X_train = _pickle.load(file)
file.close()

In [4]:
file = open('y_train.txt', 'rb')
y_train = (_pickle.load(file))[:994770]
file.close()

In [3]:
from sklearn.linear_model import LogisticRegression

In [5]:
LogReg = LogisticRegression(class_weight='balanced', solver='saga', n_jobs=4)
LogReg.fit(X_train, y_train)

/home/nariman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=4, penalty='l2', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [6]:
file = open('LogReg_FT.model', 'wb')
_pickle.dump(LogReg, file)
file.close()

In [15]:
file = open('LogReg_FT.model', 'rb')
LogReg = _pickle.load(file)
file.close()

In [ ]:
LogReg.predict(X)

In [16]:
X_test = np.empty((0,200))
for i in range(10):
    print(i)
    file = open ('FF_emb_test'+str(i)+'.dat', 'rb')
    adder = _pickle.load(file)
    X_test = np.vstack((X_test, adder))
    file.close()

0
1
2
3
4
5
6
7
8
9


In [7]:
file = open('y_test.txt', 'rb')
y_test = (_pickle.load(file))[:427230]
file.close()

In [10]:
print(sklearn.metrics.classification_report(y_pred=LogReg.predict(X_test), y_true=y_test))

              precision    recall  f1-score   support

           0       0.99      0.92      0.96    406751
           1       0.36      0.88      0.52     20479

   micro avg       0.92      0.92      0.92    427230
   macro avg       0.68      0.90      0.74    427230
weighted avg       0.96      0.92      0.94    427230



Разумеется нехорошо мерить тот же скор на меньших выборках, но у нас нет выбора.
Попробуем обучить еще один классификатор на других данных и замиксить.

In [11]:
file = open('LogReg_FT_1.model', 'wb')
_pickle.dump(LogReg, file)
file.close()

In [10]:
file = open('LogReg_FT.model', 'rb')
model_1 = _pickle.load(file)
file.close()

In [5]:
model_1.predict(X_)

In [24]:
X_train = np.empty((0,200))
for i in range(10, 20):
    print(i-10)
    file = open ('FF_emb_train'+str(i)+'.dat', 'rb')
    adder = _pickle.load(file)
    X_train = np.vstack((X_train, adder))
    file.close()
    
file = open('X_train_10_20.dat', 'wb')
_pickle.dump(X_train, file)
file.close()

0
1
2
3
4
5
6
7
8
9


In [ ]:
file = open('X_train_10_20.dat', 'rb')
X_train = _pickle.load(file)
file.close()

In [31]:
file = open('y_train.txt', 'rb')
y_train = (_pickle.load(file))[994770:1989540]
file.close()

In [32]:
LogReg_2 = LogisticRegression(class_weight='balanced', solver='saga', n_jobs=4)
LogReg_2.fit(X_train, y_train)
file = open('LogReg_FT_2.model', 'wb')
_pickle.dump(LogReg_2, file)
file.close()

/home/nariman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [4]:
file = open('LogReg_FT_2.model', 'rb')
model_2 = _pickle.load(file)
file.close()

In [8]:
file = open('LogReg_FT.model', 'rb')
model_1 = _pickle.load(file)
file.close()

In [9]:
file = open('LogReg_FT_1.model', 'wb')
_pickle.dump(model_1,file)
file.close()

In [8]:
X_test = np.empty((0,200))
for i in range(20):
    print(i)
    file = open ('FF_emb_test'+str(i)+'.dat', 'rb')
    adder = _pickle.load(file)
    X_test = np.vstack((X_test, adder))
    file.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [23]:
X_test.shape

(854460, 200)

In [9]:
file = open('y_test.txt', 'rb')
y_test = (_pickle.load(file))[:854460]
file.close()

In [10]:
print(sklearn.metrics.classification_report(y_pred=model_2.predict(X_test), y_true=y_test))

              precision    recall  f1-score   support

           0       0.99      0.92      0.96    813593
           1       0.36      0.88      0.51     40867

   micro avg       0.92      0.92      0.92    854460
   macro avg       0.68      0.90      0.74    854460
weighted avg       0.96      0.92      0.94    854460



In [21]:
print(sklearn.metrics.classification_report(y_pred=[(x[0]+x[1] > 0)*1 for x in zip(model_1.predict(X_test), model_2.predict(X_test))], y_true=y_test))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95    813593
           1       0.34      0.89      0.50     40867

   micro avg       0.91      0.91      0.91    854460
   macro avg       0.67      0.90      0.72    854460
weighted avg       0.96      0.91      0.93    854460



Не получилось. А может деревом?

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
trees = RandomForestClassifier(n_estimators=40, max_depth=15, n_jobs=4, class_weight='balanced')
trees.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=15, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=40, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [11]:
print(sklearn.metrics.classification_report(y_pred=trees.predict(X_test), y_true=y_test))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97    813593
           1       0.44      0.89      0.59     40867

   micro avg       0.94      0.94      0.94    854460
   macro avg       0.72      0.92      0.78    854460
weighted avg       0.97      0.94      0.95    854460



In [12]:
file = open('Random_Forest_FT_1.model', 'wb')
_pickle.dump(trees,file)
file.close()

In [13]:
file = open('X_train_10_20.dat', 'rb')
X_train = _pickle.load(file)
file.close()
file = open('y_train.txt', 'rb')
y_train = (_pickle.load(file))[994770:1989540]
file.close()

In [15]:
trees_2 = RandomForestClassifier(n_estimators=40, max_depth=15, n_jobs=4, class_weight='balanced')
trees_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=15, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=40, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [16]:
file = open('Random_Forest_FT_2.model', 'wb')
_pickle.dump(trees_2,file)
file.close()

In [17]:
file = open('Random_Forest_FT_1.model', 'rb')
trees_1 = _pickle.load(file)
file.close()

In [18]:
print(sklearn.metrics.classification_report(y_pred=[(x[0]+x[1] > 0)*1 for x in zip(trees_1.predict(X_test), trees_2.predict(X_test))], y_true=y_test))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96    813593
           1       0.39      0.91      0.54     40867

   micro avg       0.93      0.93      0.93    854460
   macro avg       0.69      0.92      0.75    854460
weighted avg       0.97      0.93      0.94    854460



Ну что ж, будем пользоваться бейслайном.

Итог: воспользуемся второй моделью, обученной на всех словах, сохраним энкодер и саму модель.

TODO

Что хочется сделать:  
1. Улучшить предобработку слов. Вставить лематизацию и нормальное удаление стоп-слов.  
2. Добавить фичей получше. К примеру очень хочется использовать контекст: предыдущее и последующее слова. Так мы сможем обнаруживать названия препаратов, которые очень сильно не похожи на остальные(возможно название бренда).
3. Использовать все данные. В данный момент, даже 30% трейна занимают почти 4гб в пикле, а есть еще тест, который в 2 раза меньше. Как это можно сделать: уменьшить размер эмбединга фасттекста, использовать больше оперативной памяти, либо же использовать смесь моделей, обученных на разных частях данных, желательно сильно пересекающихся.
Можно подумать, что применить стандартные методы понижения размерности (PCA, SVD и тд) - неплохая идея, но на NLP они, как правило, не работают.